# Code to convert the excel file to a usable json

In [ ]:
from flask import Flask, render_template, jsonify
from flask_pymongo import PyMongo
import json
import ev_ranges

def ranges():
    # read in the excel file dropping the top junk rows
    ev_ranges_file = "Data/10963_EV_range_efficiency_1-6-22.xlsx"
    ev_ranges_df = pd.read_excel(ev_ranges_file, skiprows=2)

    # Removing unused data columns

    ev_ranges_df.drop(ev_ranges_df.columns[[0,5,6,7,8,9]], axis=1, inplace=True)


    # Removes the trailing NaNs
    ev_ranges_df.dropna(inplace=True)

    # Group by year with mean of ranges then return to a dataframe format
    ev_gbymean = ev_ranges_df.groupby("Model Year").mean().reset_index()

    # Group by year with max of ranges then return to a dataframe format
    ev_gbymax = ev_ranges_df.groupby("Model Year")["Range"].max().reset_index()

    # Merge the two grouped dataframes back to one
    ev_ranges_by_year = pd.merge(ev_gbymean,ev_gbymax, how="left", on="Model Year")

    # Rename the columns
    ev_ranges_by_year.rename(columns={'Range_x' : 'Average Range', 'Range_y' : 'Max Range'}, inplace=True)

    # Remove first two rows for continuous years
    ev_ranges_by_year.drop([0,1], inplace=True)

    ev_ranges_by_year.reset_index(drop=True, inplace=True)
    
    ev_ranges_by_year.reset_index(drop=True, inplace=True)

    # Return final dataframe as a json
    return ev_ranges_by_year.to_dict('split')

In [ ]:
# Create an instance of Flask
app = Flask(__name__)

In [ ]:
# Use PyMongo to establish Mongo connection
conn = "mongodb://localhost:27017/ev_data"
mongo = PyMongo(app, uri = conn)

In [ ]:
ev_range = ranges()

In [ ]:
ev_range

In [ ]:
type(ev_range)

In [ ]:
mongo.db.ranges.update_one({}, {"$set": ev_range}, upsert=True)

In [ ]:
result = mongo.db.ranges.find()

In [ ]:
result_list = list(result)

In [ ]:
result

In [ ]:
if len(result_list) > 0:
        # Return the first result only and strip off the '_id'
        data = result_list[0] 
        id_to_discard = data.pop('_id', None)
else:
        # Construct an error message
        data = {'Error': 'No data found'} 

In [ ]:
data